# Módulo 4: La Especialización (Boosting - XGBoost)

## Objetivo Didáctico
Entender la diferencia entre **Bagging** (Random Forest) y **Boosting** (XGBoost). Aprenderemos cómo los modelos pueden aprender de sus propios errores.

## 1. La Analogía: El Estudiante y el Tutor
*   **Random Forest (Bagging)**: 100 estudiantes hacen el examen por separado. Promediamos sus notas.
*   **XGBoost (Boosting)**:
    1.  El Estudiante 1 hace el examen. Se equivoca en las preguntas difíciles.
    2.  El Estudiante 2 estudia **solo las preguntas que falló el Estudiante 1**.
    3.  El Estudiante 3 estudia los errores del Estudiante 2.
    4.  Al final, sumamos el conocimiento de todos.

Esto lo hace mucho más preciso, pero también más propenso a "memorizar" demasiado (Overfitting) si no tenemos cuidado.

In [1]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Cargar datos
df = pd.read_csv('../../data/processed/clean_student_data.csv')
df_binary = df[df['Target'].isin(['Dropout', 'Graduate'])].copy()
df_binary['Target_Binary'] = df_binary['Target'].apply(lambda x: 1 if x == 'Dropout' else 0)

features = [
    'Curricular units 1st sem (grade)', 
    'Age at enrollment', 
    'Tuition fees up to date', 
    'Scholarship holder', 
    'Debtor'
]

X = df_binary[features]
y = df_binary['Target_Binary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

## 2. Entrenando al Campeón (XGBoost)
XGBoost es el algoritmo que gana la mayoría de las competencias de Kaggle con datos tabulares.

In [2]:
xgb_model = xgb.XGBClassifier(
    n_estimators=100, 
    learning_rate=0.1, 
    max_depth=3, 
    use_label_encoder=False, 
    eval_metric='logloss',
    random_state=42
)

xgb_model.fit(X_train, y_train)
xgb_acc = accuracy_score(y_test, xgb_model.predict(X_test))
print(f"Precisión de XGBoost: {xgb_acc:.2%}")

Precisión de XGBoost: 83.01%


/opt/homebrew/Caskroom/miniconda/base/lib/python3.13/site-packages/xgboost/training.py:199: UserWarning: [09:59:18] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


## 3. Comparación de Titanes
¿Quién ganó en este caso simple?

In [3]:
print(f"XGBoost: {xgb_acc:.2%}")
# (Recuerda el valor de RF del notebook anterior, suele ser similar o ligeramente menor en datasets simples)
print("Random Forest: ~ (Valor del notebook anterior)")

XGBoost: 83.01%
Random Forest: ~ (Valor del notebook anterior)


## 4. El Dilema de la Interpretabilidad
XGBoost es increíblemente potente. Puede encontrar patrones sutiles que a un humano se le escaparían.

**Pero...** sigue siendo una Caja Negra. Sabemos QUÉ predice, pero es difícil saber POR QUÉ para un alumno específico.

¿O no? En el siguiente módulo, romperemos esa caja con **SHAP**.